In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torchvision
from tqdm import tqdm
import pickle
np.random.seed(0)

# Generate similar dataset (CIFAR10/100, TinyImagenet)

In [53]:
dataset = "TinyImagenet"
data_dir = os.path.join("./", dataset)
print('Data stored in %s' % data_dir)
if dataset == "CIFAR10":
    trainset = torchvision.datasets.CIFAR10(root=data_dir, train=True, download=True, transform=None)
elif dataset == "CIFAR100":
    trainset = torchvision.datasets.CIFAR100(root=data_dir, train=True, download=True, transform=None)
else:
    trainset = torchvision.datasets.ImageFolder('tiny-imagenet-200/train/') # 
# Load train set
X_list = []
y_list = []
# recover from np.array
# from PIL import Image
# im = Image.fromarray(X_train_list[0])
for data in tqdm(trainset):
    x, y = data
    X_list.append(np.array(x))
    y_list.append(y)

# get victim sub-dataset and attacker sub-dataet 
vic_num = len(trainset) // 2
# np.random.seed(0)
# vic_idx = np.random.choice(len(trainset), vic_num, replace=False) 
# equal to np.random.shuffle() and then [:vic_num], therefore the vic_idx here is the same as previous one
# print(vic_idx) 
np.random.seed(0)
train_idx_array = np.arange(len(trainset))
np.random.shuffle(train_idx_array)
vic_idx = train_idx_array[:vic_num]

intersect_proportion = 1.0
shift = int(intersect_proportion * vic_num)
start_att_idx = vic_num - shift
att_idx = train_idx_array[start_att_idx: start_att_idx + vic_num]
print()
print(train_idx_array)
print(start_att_idx, start_att_idx + vic_num)

Data stored in ./TinyImagenet
100%|██████████| 100000/100000 [00:12<00:00, 8047.06it/s]
[ 3582 60498 53227 ... 42613 43567 68268]
0 50000



the index should be 

CIFAR100: 11841 19602 45519 ... 42613 43567  2732

TinyImagenet: 3582 60498 53227 ... 42613 43567 68268


In [54]:
X_np = np.array(X_list)
y_np = np.array(y_list)

In [55]:
X_set1 = X_np[vic_idx]
y_set1 = y_np[vic_idx]
X_set2 = X_np[att_idx]
y_set2 = y_np[att_idx]

In [56]:
pickle.dump(((X_set1, y_set1), (X_set2, y_set2)), open("./similarity/{}_intersect_{}.pkl".format(dataset, intersect_proportion), "wb"))
# tuple((X_set1 / 255).mean(axis=(0, 1, 2)))
# (X_set1 / 255).std(axis=(0, 1, 2))